<center><h1>FAULTY AIR QUALITY PREDICTION</h1>

### Importing of the neccessary libraries for the project

In [ ]:
import pandas as pd
import numpy as np
import warnings,os
warnings.filterwarnings("ignore")
pd.set_option("display.max_rows",None)
from collections import Counter
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading of the CSV datasets

In [ ]:
train = 'drive/MyDrive/FaultyAirQualityPrediction/train.csv'
test = 'drive/MyDrive/FaultyAirQualityPrediction/test.csv'
df = pd.read_csv(train,parse_dates = ['Datetime']).drop(["ID"],axis=1)
test = pd.read_csv(test,parse_dates = ['Datetime'])
ID = test.iloc[:,0]
test = test.drop("ID",axis=1)

In [ ]:
df.head()

,Datetime,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Offset_fault
0,2021-11-03 04:06:31,52.58,49.52,17.4,96.0,0
1,2021-11-08 18:43:23,35.25,33.40,25.0,75.0,0
2,2021-11-07 09:50:33,19.18,23.50,24.9,75.0,0
3,2022-01-01 18:55:15,19.40,15.48,24.9,70.0,0
4,2021-11-05 22:23:48,38.30,34.77,20.9,89.0,0


Spliting the data into the training and the validatio set effectively before preprocessing them 

In [ ]:
from sklearn.model_selection import train_test_split as tts
train_df,test_df = tts(df,test_size=0.2,random_state=43)

### Data Preprocessing

In [ ]:
def process(data):
    data['Day'] = data.Datetime.dt.day
    data['Month'] = data.Datetime.dt.month
    data['Year'] = data.Datetime.dt.year
    data['Hour'] = data.Datetime.dt.hour
    data = data.drop("Datetime",axis=1)
    return data
train_df = process(train_df)
test_df = process(test_df)
test = process(test)

In [ ]:
train_df.head()

,Sensor1_PM2.5,Sensor2_PM2.5,Temperature,Relative_Humidity,Offset_fault,Day,Month,Year,Hour
258598,36.87,39.15,21.4,84.0,0,30,11,2021,20
211230,52.03,42.82,21.6,80.0,1,26,10,2021,20
231547,25.12,19.92,20.6,89.0,1,23,12,2021,4
64276,11.92,7.28,27.2,66.0,1,10,12,2021,17
182564,16.65,16.97,27.9,51.0,0,12,12,2021,10


In [ ]:
def fillnull(data):
    for i in data:
        data[i] = data[i].fillna(-500)
    return data
train_df = fillnull(train_df)
test_df = fillnull(test_df)
test = fillnull(test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def scalling(data):
    scaler = MinMaxScaler()
    scaler.fit_transform(data)
    scaled_data = pd.DataFrame(data,columns=data.columns)
    return scaled_data
train_df = scalling(train_df)
test_df = scalling(test_df)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
def split(X,y):
    X_ = X.drop(y,axis=1)
    y_ = X[y]
    ros = RandomOverSampler(random_state=2020)
    X,y = ros.fit_resample(X_,y_)
    return X,y
X_train,y_train = split(train_df,"Offset_fault")
X_test,y_test = split(test_df,"Offset_fault")

### Cross Validation For Model Selection

In [ ]:
from lightgbm.compat import LGBMDeprecationWarning
from lightgbm.sklearn import LGBMModel
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
from sklearn.svm import SVC
sv = SVC()
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier()
from xgboost import XGBClassifier
xg = XGBClassifier(eval_metric='logloss')
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
from catboost import CatBoostClassifier
cb = CatBoostClassifier(silent=True)
models = {
    "Model":[rf,xg,lgb,cb],
    "Name":["RandomForest","XGBBoost","LGBM","CatBoost"]
}

In [ ]:
sk = StratifiedKFold(n_splits=5)
cnt = []
for model,name in tqdm(np.array(pd.DataFrame(models))):
    cvs = cross_val_score(estimator=model,X=X_train,y=y_train,cv=sk)
    score = np.mean(cvs)
    cnt.append([name,score])
cnt = pd.DataFrame(cnt)
mdls = cnt.rename({0:"Model",1:"Score"},axis=1)
mdls.sort_values(by="Score",ascending=False)

100%|██████████| 4/4 [06:44<00:00, 101.07s/it]


,Model,Score
0,RandomForest,0.981494
3,CatBoost,0.962059
2,LGBM,0.944609
1,XGBBoost,0.922057


In [ ]:
from sklearn.metrics import accuracy_score
model_dc = []
for model,name in tqdm(np.array(pd.DataFrame(models))):
    model.fit(X_train,y_train)
    pred = model.predict(X_test)
    score = accuracy_score(y_test,pred)
    model_dc.append([name,score])
model_df = pd.DataFrame(model_dc).rename({0:"Model",1:"Score"},axis=1)
model_df.sort_values(by="Score",ascending=False)

100%|██████████| 4/4 [01:41<00:00, 25.42s/it]


,Model,Score
0,RandomForest,0.968643
3,CatBoost,0.960763
1,XGBBoost,0.945487
2,LGBM,0.923352


Making the prediction On Validation Set

In [ ]:
model = LGBMClassifier(
    n_estimators=1500,
    random_state=100
)
model.fit(X_train,y_train)
pred = model.predict(X_test)
print("Accuracy Score: ",accuracy_score(y_test,pred))

Accuracy Score:  0.9684278350515464


In [ ]:
predict = model.predict(test)
subm = {"ID":ID,"Offset_fault":predict}
subm = pd.DataFrame(subm)
subm.to_csv("submision.csv",index=False)